# Notebook with test code

---

## Imports

### Standard library imports

In [1]:
import yaml

import sys

from datetime import (date, datetime)

import os

import pickle

import re

### Third party libraries

In [2]:
import boto3

import pandas as pd

### Local application imports

In [3]:
sys.path.append("../")

In [4]:
%load_ext autoreload
%autoreload 2

from src.etl.ingesta_almacenamiento import (
    
    ## Functions
    get_client,
    ingesta_inicial,
    ingesta_consecutiva,
    get_s3_resource,
    request_data_to_API,
    get_s3_credentials,
    
)


from src.utils.utils import (

    get_api_token

)


from src.utils.data_dict import (
    data_dict
)

from src.utils.params_ml import(
    models_dict
)

---

## Configuration

In [5]:
s3_creds = get_s3_credentials("../conf/local/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

## AWS base examples

### S3

#### Initial configuration

##### Resource

##### Client

In [ ]:
s3_creds = get_s3_credentials("../conf/local/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

#### Connecting to existing bucket and viewing contents

In [ ]:
bucket = "data-product-architecture-equipo-9"
key ="ingestion/consecutive"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
objects

#### Reading pickle from S3

In [ ]:
obj_path = [file["Key"] for file in objects if "2021-03-18" in file["Key"]][0]
obj_path

In [ ]:
response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)
response

In [ ]:
body = response["Body"].read()

---

In [ ]:
pickle.loads(pickle.loads(body))

#### Creating bucket

In [ ]:
bucket_name = "comdline-test-bucket-rob"

s3.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={'LocationConstraint': 'us-west-2'},
    ACL="private"
)

#### Uploading file to bucket

In [ ]:
file_to_upload = "../../admin/test_file3_for_s3.txt"
file_name = file_to_upload.split(sep="/")[-1]

In [ ]:
# s3.upload_file(file_to_upload, bucket_name, file_name)

s3.meta.client.upload_file(file_to_upload, bucket_name, file_name)

In [ ]:
[obj for obj in s3.Bucket(bucket_name).objects.all()]

#### Downloading files from bucket

In [ ]:
path_to_download = "/Users/rp_mbp/Documents/ReposRob_RobPer/ITAMmcd/semestre_2/Arquitectura_Prod_Dat/Venv_ArqPD/repos/dpa_2021/_robdir/" + file_name

s3.meta.client.download_file(bucket_name, file_name, path_to_download)

#### Deleting bucket files

In [ ]:
s3.Object(bucket_name, file_name).delete()

---

#### Deleting bucket

## Downloading and storing Chicago data

#### Interacting with API

In [ ]:
dataset_id = "4ijn-s7e5"

In [ ]:
token = get_api_token("../conf/local/credentials.yaml")
token

In [ ]:
client = get_client(token)
client

In [ ]:
most_rec_date = '2021-04-10'

In [ ]:
soql_query = "inspection_date >= '{}'".format(most_rec_date)
soql_query

In [ ]:
x = client.get(dataset_id, 
               limit=10,
               where=soql_query
              )

In [ ]:
x

In [ ]:
len(x)

In [ ]:
xx = pd.DataFrame(x)
xx

In [ ]:
str(xx.shape)

##### Using "dumps"

In [ ]:
xx = pickle.dumps(x)

In [ ]:
xx

In [ ]:
pickle.loads(xx)

##### Using "dump"

In [ ]:
pkl_store_loc = !pwd
pkl_store_loc = pkl_store_loc[0] + "/prueba_pickle.pkl"

In [ ]:
pkl_store_loc

In [ ]:
pickle.dump(x, open(pkl_store_loc, 'wb'))

In [ ]:
pickle.load(open(pkl_store_loc, "rb"))

##### Storing data in S3

In [ ]:
pkl_store_loc = !pwd
pkl_store_loc = pkl_store_loc[0] + "/prueba_pickle.pkl"
pkl_store_loc

In [ ]:
## Storing file locally
pickle.dump(x, open(pkl_store_loc, "wb"))

In [ ]:
## Loading file from local and saving as variable
xx = pickle.dumps(pickle.load(open(pkl_store_loc, "rb")))

In [ ]:
## Saving pickle in s3
s3.put_object(
    Bucket=bucket_name,
    Key="test_pickle.pkl",
    Body=xx
)

##### Downloading data from s3 and unpickling

In [ ]:
res_xx = s3.get_object(
    Bucket=bucket_name,
    Key="test_pickle.pkl"
)
res_xx

In [ ]:
body = res_xx["Body"].read()

In [ ]:
pickle.loads(body)

#### Uploading info

#### Checking aws s3 contents

In [ ]:
s3 = get_s3_resource()

In [ ]:
cont_ingest_path

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=cont_ingest_path)['Contents']
objects

In [ ]:
objects.append(
 {'Key': 'ingestion/consecutive/consecutive-inspections-2021-02-22.pkl',
  'LastModified': "datetime.datetime(2021, 2, 22, 2, 53, 54, tzinfo=tzutc())",
  'ETag': '"79cff7864a646f1dfd4d51b4e732a226"',
  'Size': 1152660,
  'StorageClass': 'STANDARD'}
)

objects.append(
 {'Key': 'ingestion/consecutive/consecutive-inspections-2021-02-15.pkl',
  'LastModified': "datetime.datetime(2021, 2, 22, 2, 53, 54, tzinfo=tzutc())",
  'ETag': '"79cff7864a646f1dfd4d51b4e732a226"',
  'Size': 1152660,
  'StorageClass': 'STANDARD'}
)

objects.append(
 {'Key': 'ingestion/consecutive/consecutive-inspections-2021-02-10.pkl',
  'LastModified': "datetime.datetime(2021, 2, 22, 2, 53, 54, tzinfo=tzutc())",
  'ETag': '"79cff7864a646f1dfd4d51b4e732a226"',
  'Size': 1152660,
  'StorageClass': 'STANDARD'}
)

In [ ]:
rex = str(cont_dat_prefix) + "(.*).pkl"

lx = [datetime.strptime(re.search(rex, obj["Key"]).group(1), '%Y-%m-%d') for obj in objects if cont_dat_prefix in obj["Key"]]
most_rec_date = datetime.strftime(max(lx), '%Y-%m-%d')
most_rec_date

In [ ]:
s3x = pickle.loads(s3.get_object(Bucket=bucket_name, Key=(cont_ingest_path + "consecutive-inspections-2021-02-21.pkl"))['Body'].read())

In [ ]:
dfx = pd.DataFrame(s3x)
dfx

In [ ]:
dfx["inspection_date"] = pd.to_datetime(dfx["inspection_date"])

In [ ]:
print("max date: ", max(dfx["inspection_date"]))
print("min date: ", min(dfx["inspection_date"]))

In [ ]:
dfx.groupby(pd.Grouper(key="inspection_date")).count()[["inspection_id"]]

---

## Creating local directories for temporal data

In [ ]:
base_path = "../src/pipeline/luigi/ingestion_tmp/"

### Find most recent ingestion

#### Case 1: There is previous consecutive downloads

In [ ]:
lyrs = [ydir[-4:] for ydir in os.listdir(base_path + "consecutive") if "YEAR=" in ydir]
lyrs

In [ ]:
mr_yr = max(lyrs)
mr_yr

In [ ]:
lmths = [mdir[-2:] for mdir in os.listdir(base_path + "consecutive" + "/" + "YEAR=" + mr_yr) if "MONTH=" in mdir]
lmths

In [ ]:
mr_mth = max(lmths)
mr_mth

In [ ]:
lings = [ing for ing in os.listdir(base_path + "consecutive" + "/" + "YEAR=" + mr_yr + "/" + "MONTH=" + mr_mth)]
lings

In [ ]:
regex = "consecutive_inspections_" + "(.*).pkl"

In [ ]:
min([re.search(regex, ing).group(1) for ing in lings])

In [ ]:
lx = ['.DS_Store', 'consecutive_inspections_2021-03-05.pkl', 'consecutive_inspections_2021-03-10.pkl']
lx

In [ ]:
cont_dat_prefix = "consecutive_inspections_"

In [ ]:
regex = cont_dat_prefix + "(.*).pkl"

In [ ]:
lx

In [ ]:
lings

In [ ]:
most_recent_ing = max([re.search(regex, ing).group(1) for ing in lx if ".pkl" in ing])
most_recent_ing

#### Additional notes

In [ ]:
x = date.today().strftime('%Y-%m-%d')

In [ ]:
x[5:7]

---

## Creating .csv files with metadata

### Working with previous metadata pickle

#### Transformation pickle

In [ ]:
pkl_meta = "../results/metadata/transform_metadata.pkl"

In [ ]:
dfx = pickle.load(open(pkl_meta, "rb"))
dfx

In [ ]:
mx = [dfx.index[0]]
mx

In [ ]:
[mx.append(val) for val in dfx.iloc[0, :]]

In [ ]:
mx

In [ ]:
## Overwriting csv file from a dataframe (no header)
def write_csv_from_df(df, filepath, filename):
    
    ## Extracting df contents as list
    mdata_list = [df.index[0]]
    [mdata_list.append(val) for val in df.iloc[0, :]]
    
    ## Creating and writing csv file with extracted list
    with open(filepath + filename, mode="w") as metadata_file:
        metadata_writer = csv.writer(metadata_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        metadata_writer.writerow(mdata_list)

In [ ]:
filepath = "metadata_test_dir/"

In [ ]:
filename = "transformation_metadata_2.csv"

In [ ]:
write_csv_from_df(dfx, filepath, filename)

In [ ]:
dfx

#### Model selection pickle

In [ ]:
s3_creds = get_s3_credentials("../conf/local/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

In [ ]:
bucket = "data-product-architecture-equipo-9"
key ="model_selection"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
objects

In [ ]:
obj_path = [file["Key"] for file in objects if "trained" in file["Key"]][0]
obj_path

In [ ]:
response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)
response

In [ ]:
body = response["Body"].read()

In [ ]:
model = pickle.loads(body)
model

In [ ]:
str(model["best_trained_model"])

In [ ]:
model["best_trained_model"]

### Creating dummy metadata

#### Line 1

In [ ]:
meta_1 = str(datetime.now())

In [ ]:
meta_2 = str(10)

In [ ]:
meta_3 = "carnegie"

In [ ]:
meta_comp = ",".join([meta_1, meta_2, meta_3])
meta_comp

In [ ]:
import csv
with open("transformation_metadata.csv", mode="w") as metadata_file:
    metadata_writer = csv.writer(metadata_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
    metadata_writer.writerow([meta_1, meta_2, meta_3])

#### Creating string based on dictionary keys for metadata

In [ ]:
" | ".join([mdl for mdl in models_dict])

## Saving dictionary of dataframes as pickle

### Creating dictionary of dataframes

In [ ]:
dfx1 = {
    "col1_df1": list(range(1, 11)),
    "col2_df1": list("htkdlekdjc"),
}
dfx1 = pd.DataFrame.from_dict(dfx1)
dfx1

In [ ]:
dfx2 = {
    "col1_df2": list(range(11, 21)),
    "col2_df2": list("jfldurytgc"),
}
dfx2 = pd.DataFrame.from_dict(dfx2)
dfx2

In [ ]:
df_dict = {
    "dfx1": dfx1,
    "dfx2": dfx2
}
df_dict

In [ ]:
df_dict['dfx2']

### Storing dictionary as pickle variable and unpickling

In [ ]:
df_dict_pkl = pickle.dumps(df_dict)

In [ ]:
df_dict_x = pickle.loads(df_dict_pkl)

In [ ]:
df_dict_x['dfx2']

### Storing dictionary as pickle in local memory and unpickling

In [ ]:
pkl_file = "../data/pickles/test_df_dict_pkl.pkl"

In [ ]:
pickle.dump(df_dict, open(pkl_file, "wb"))

In [ ]:
df_dict_x = pickle.load(open(pkl_file, "rb"))

In [ ]:
df_dict_x['dfx2']

---

## Inspecting results

### From local pickles

#### Data ingested

In [ ]:
pth = "../data/raw/Food_Inspections.csv"

In [ ]:
dfx = pd.read_csv(pth)

In [ ]:
dfx

#### Extraction dataframe

In [ ]:
pth = "../data/pickles/ingest_df.pkl"

In [ ]:
dfx = pickle.load(open(pth, "rb"))

In [ ]:
dfx

#### Transformation df

In [ ]:
pth = "../data/pickles/transformation_df.pkl"

In [ ]:
dfx = pickle.load(open(pth, "rb"))

In [ ]:
dfx

#### Feature engineering picke

In [ ]:
pth = "../data/pickles/fe_results.pkl"

In [ ]:
fe_results = pickle.load(open(pth, "rb"))

In [ ]:
fe_results["df_imp_engineered_features"]

In [ ]:
fe_results["data_labels"]

In [ ]:
fe_results["ohe_reference"]

#### Models training pickle

In [ ]:
pth = "../data/pickles/mt_results.pkl"

In [ ]:
mt_results = pickle.load(open(pth, "rb"))

In [ ]:
mt_results["training_labels"]

In [ ]:
mt_results["test_labels"]

#### Model selection pickle

In [ ]:
pth = "../data/pickles/ms_results.pkl"

In [ ]:
ms_results = pickle.load(open(pth, "rb"))

In [ ]:
ms_results["model_test_predict_labels"]

In [ ]:
ms_results["model_test_predict_scores"]

### From AWS pickles

#### Ingestion

In [7]:
bucket = "data-product-architecture-equipo-9"
key ="ingestion/"

In [8]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][2]
# obj_path = [file["Key"] for file in objects][2]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [9]:
ing_results = pickle.loads(response["Body"].read())

In [10]:
ing_results

,facility-type,risk,city,zip,inspection-type,results,violations,label
inspection-id,,,,,,,,
2509401,grocery_store,risk_1_-high-,chicago,60632,canvass_re-inspection,pass,nan,1
2509456,restaurant,risk_1_-high-,chicago,60642,canvass_re-inspection,pass,37._food_properly_labeled-_original_container_...,1
2509420,restaurant,risk_1_-high-,chicago,60641,license_re-inspection,pass,51._plumbing_installed-_proper_backflow_device...,1
2509377,grocery_store,risk_2_-medium-,chicago,60625,canvass,fail,11._food_obtained_from_approved_source_-_comme...,0
2509345,grocery_store,risk_3_-low-,chicago,60626,license,fail,5._procedures_for_responding_to_vomiting_and_d...,0
...,...,...,...,...,...,...,...,...
70269,restaurant,risk_1_-high-,chicago,60634,license_re-inspection,pass,nan,1
104236,restaurant,risk_1_-high-,chicago,60611,canvass,fail,18._no_evidence_of_rodent_or_insect_outer_open...,0
52234,restaurant,risk_1_-high-,chicago,60657,license_re-inspection,pass,nan,1


In [11]:
ing_results["facility-type"].value_counts()

restaurant                      146544
grocery_store                    28563
school                           13567
nan                               4912
children-s_services_facility      3824
                                 ...  
hooka_bar                            1
internet_cafe                        1
health_food_store                    1
alternative_school                   1
produce_vendor                       1
Name: facility-type, Length: 448, dtype: int64

#### Transformation

In [54]:
bucket = "data-product-architecture-equipo-9"
key ="transformation/"

In [55]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][0]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [56]:
tra_results = pickle.loads(response["Body"].read())

In [57]:
tra_results

,facility-type,risk,city,zip,inspection-type,results,violations,label,serious_violations,zip-income-class
inspection-id,,,,,,,,,,
2509401,facility-type_other,risk_1_-high-,chicago,60632,canvass,pass,-_nan,1,no_result,Medium
2509456,restaurant_bar,risk_1_-high-,chicago,60642,canvass,pass,-_37._food_properly_labeled-_original_containe...,1,serious_violations,Other
2509420,restaurant_bar,risk_1_-high-,chicago,60641,license,pass,-_51._plumbing_installed-_proper_backflow_devi...,1,no_serious_violations,Medium
2509397,facility-type_other,risk_1_-high-,chicago,60605,license,no_entry,-_nan,0,no_result,High
2509328,facility-type_other,risk_2_-medium-,chicago,60613,canvass,out_of_business,-_nan,0,no_result,High
...,...,...,...,...,...,...,...,...,...,...
52234,restaurant_bar,risk_1_-high-,chicago,60657,license,pass,-_nan,1,no_result,High
67732,restaurant_bar,risk_1_-high-,chicago,60613,license,pass,-_nan,1,no_result,High
67757,restaurant_bar,risk_2_-medium-,chicago,60601,inspection-type_other,pass,-_nan,1,no_result,High


In [58]:
tra_results["zip-income-class"].value_counts()

Medium    78578
Low       72635
High      61494
Other      6994
Name: zip-income-class, dtype: int64

#### Feature engineering

In [34]:
bucket = "data-product-architecture-equipo-9"
key ="feature_engineering/"

In [35]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][0]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [36]:
fe_results = pickle.loads(response["Body"].read())

In [48]:
tst_ids = fe_results["data_labels"].index

In [42]:
tst_feats = fe_results["df_imp_engineered_features"]

In [44]:
tst_feats.toarray()

array([[0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

#### Trained models

In [6]:
bucket = "data-product-architecture-equipo-9"
key ="trained_models/"

In [7]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][0]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [8]:
objects

[{'Key': 'trained_models/trained_models_2021-05-12.pkl',
  'LastModified': datetime.datetime(2021, 5, 12, 2, 39, 16, tzinfo=tzutc()),
  'ETag': '"d8cbb8cb84d97da413cfc1f2b7af65c7"',
  'Size': 132539,
  'StorageClass': 'STANDARD'}]

In [ ]:
mt_results = pickle.loads(response["Body"].read())

In [ ]:
dfx = mt_results["test_labels"].to_frame()

In [ ]:
dfx.reset_index(inplace=True)

In [ ]:
dfx

#### Models selection

In [9]:
bucket = "data-product-architecture-equipo-9"
key ="model_selection/"

In [30]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][-1]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [31]:
ms_results = pickle.loads(response["Body"].read())

In [33]:
sel_model = ms_results["best_trained_model"]

In [40]:
tst_data

<1000x18 sparse matrix of type '<class 'numpy.float64'>'
	with 5000 stored elements in Compressed Sparse Row format>

In [49]:
len(tst_ids)

1000

In [51]:
tst_pred = sel_model.predict(tst_feats)

In [56]:
dfx = pd.DataFrame.from_dict(
    {
        "ids": tst_ids,
        "prediction": tst_pred,
    }
)

dfx.set_index("ids", inplace=True)

##### Making some predictions 

#### Bias and fairness

In [6]:
bucket = "data-product-architecture-equipo-9"
key ="bias_fairness/"

In [7]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][0]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [8]:
bf_results = pickle.loads(response["Body"].read())

In [9]:
bf_results

,label_value,score,reference_group
0,1,1,High
1,1,1,Low
2,1,1,Low
3,1,1,Medium
4,1,1,Low
...,...,...,...
65906,1,1,Medium
65907,1,1,Medium
65908,1,1,Medium
65909,1,1,Medium


In [18]:
bf_results.drop("score", axis=1)

,label_value,reference_group
0,1,High
1,1,Low
2,1,Low
3,1,Medium
4,1,Low
...,...,...
65906,1,Medium
65907,1,Medium
65908,1,Medium
65909,1,Medium


In [16]:
str(bf_results.loc[3, "score"])

'1'

In [12]:
list(bf_results["reference_group"])[0]

'High'

---

## Building aequitas dataframe

### Creating initial dataframe with unique ID's and real test labels

In [ ]:
dfx = mt_results["test_labels"].to_frame()
dfx.rename(columns={"label": "test_real_labels"}, inplace=True)

### Adding the labels predicted by best model

In [ ]:
dfx["model_test_predict_labels"] = ms_results["model_test_predict_labels"]

### Adding zip and reference group

In [ ]:
rc = ["zip", "zip-income-class"]

In [ ]:
pth = "../data/pickles/transformation_df.pkl"

In [ ]:
dfxx = pickle.load(open(pth, "rb"))

In [ ]:
dfxx = dfxx.loc[:, rc]

#### Merging with labels dataframe

In [ ]:
dfx = dfx.join(dfxx, how="inner")

---

## *Notes*

---
---